In [1]:
%matplotlib inline
import requests # To read html text
from bs4 import BeautifulSoup # To parse html text
import re # Regular expressions
import operator
import matplotlib.pyplot as plt
from selenium import webdriver
import datetime
import time

In [2]:
def check_ca(string,label,info):#check the extraodinary case of camera
    pos=None
    for i in range(len(label)):
        if 'Camera' == label[i].text:
            pos=i
            break
    if pos==None:#No 'camera' row
        return None
    result=re.finditer(u'[0-9.]+ MP',info[pos].text)
    check=0
    #check if no eles in result
    if result is not None:
        result=re.finditer(u'[0-9.]+MP',info[pos].text)
    ############################
    container=[]
    for ele in result:
        container.append(ele.group())
    if u'trước' in string:
        return container[-1]
    if u'sau' in string:
        return container[0]
def find(string,label,info):
    container=[]
    substring=u''
    if u'Camera' in string:
        temp=check_ca(string,label,info)
        if temp!=None:
            return temp
        substring=string
        string=u'Độ phân giải'
        
    
    for i in range(len(label)):
        if string in label[i].text:
            container.append(info[i].text)
            
    if len(container)>0:
        if len(container)>1:
            if substring==u'Camera trước':
                try:
                    return container[2]
                except:
                    return None
            if substring==u'Camera sau':
                try:
                    return container[1]
                except:
                    return None
                
    
        return container[0]
    return 'None'

In [3]:
def day_filter(string):
    result=re.finditer(u'([0-9]+/[0-9]+/[0-9]+)|([0-9]+ ngày)|([0-9]+ năm)',string)
    for ele in result:
        return ele.group()
print day_filter(u'12 ngày')

12 ngày


In [4]:
def date2days(date):
    try:
        date=date.strip()
        if u'ngày' in date:
            return int(date.split()[0])
        if u'tháng' in date:
            return int(date.split()[0])*30
        if u'/' in date:
            date=date.split('/')
            date=datetime.date(int(date[2]),int(date[1]),int(date[0]))
            today=datetime.date.today() 
            return (today-date).days
        if u'năm' in date:
            return int(date.split()[0])*365
    except:
        return None


In [5]:
def resolution_filter(string):
    #string=u'\u0110\u1ed9 ph\xe2n gi\u1ea3i320 x 240 Pixels'
    result=re.finditer('[0-9]+ x [0-9]+',string)
    for ele in result:
        return ele.group()

def camera_filter(string):
    #string='16 MP và 8 MP'
    ite=re.finditer('([0-9.]+ MP)|[0-9.]+MP',string)
    for ele in ite:
        return ele.group()
    
def width_screen_filter(string):
    #return re.sub('[^0-9.] inch','',string)
    ite=re.finditer('[0-9.]+ inch',string)
    for ele in ite:
        return ele.group()
def CPU_filter(string):
    ite=re.finditer('[0-9.]+ (GHz|Ghz)',string)
    for ele in ite:
        return ele.group()
def memory_filter(string):
    ite=re.finditer('([0-9]+ (GB|MB))|[0-9]+(GB|MB)',string)
    for ele in ite:
        return ele.group()
def battery_filter(string):
    ite=re.finditer('[0-9]+ mAh',string)
    for ele in ite:
        return ele.group()
def name_filter(string):
    return re.sub('<([a-z]+|/|[0-9]+)>','',string)
def price_filter(string):
    return re.sub('[^0-9.]','',string)

In [6]:
def get_configuration(tree):
    dienthoai=tree.find(class_='left detail-col-left53')
    a=[s.extract() for s in dienthoai('label')]
    b=[s.extract() for s in dienthoai('span')]
    return a,b

In [7]:
def get_num_day(url):
    while(True):
        try:
            driver.find_element_by_link_text('»»').click()
        except:
            break
    time.sleep(2)
    tree = BeautifulSoup(driver.page_source)
    date=tree.find_all(class_='comment-date')
    #print day_filter(date[-1].text)
    return day_filter(date[-1].text)
def get_confi(url):
    #input: a link
    #output: a list of configurations of a phone
    

    #proceed to the last page of comment section
    driver.get(url)
    
    #get html which contains click-button to show detailed configuration of the phone
    #try if it has ordinary form
    try :
        driver.find_element_by_class_name('detail-specific-more').click()
    except:
        return
    #return [get_num_day(url)]
    #let's parse some details
    tree = BeautifulSoup(driver.page_source)
    label,info=get_configuration(tree)
    #print resolution
    resolution=find(u'Độ phân giải màn hình',label,info)
    width_screen=find(u'Màn hình',label,info)
    rear_camera=find(u'Camera sau',label,info)
    front_camera=find(u'Camera trước',label,info)
    CPU_speed=find(u'CPU',label,info)
    RAM=find(u'RAM',label,info)
    internal_storage=find(u'Bộ nhớ trong',label,info)
    battery_capacity=find(u'Dung lượng pin',label,info)
    try:
        num_day=get_num_day(url)
    except:
        num_day=None
    num_day=date2days(num_day)
    
    return [resolution_filter(resolution),
            width_screen_filter(width_screen),
            camera_filter(rear_camera),
            camera_filter(front_camera),
            CPU_filter(CPU_speed),
            memory_filter(RAM),
            memory_filter(internal_storage),
            battery_filter(battery_capacity),
            num_day]
    
#driver=webdriver.Chrome("F:\\Deadlines\\chromedriver.exe")
#get_confi('https://fptshop.com.vn/dien-thoai/iphone-7-plus-256gb')
#get_num_day('http://fptshop.com.vn/dien-thoai/wiko-u-feel-prime')

In [8]:
def get_phone_info(url):#get phone info based on Brand consist of brands, prices and configurations
    #input: a link
    #output: a list of details of a single phone
    html_text = requests.get(url).text
    tree = BeautifulSoup(html_text)
    count=0
    
    phone_list=tree.find(class_='products')
    prices=phone_list.find_all(class_='p-current-price')
    names=phone_list.find_all('h3')
    href_tree=phone_list.find_all(class_='p-item-bound')
    href=[]
    for ele in href_tree:
        _=ele.find('a')
        try:
            href.append(_['href'])
        except:
            continue
    #href=[ele.find('a')['href'] for ele in href_tree]
    print names
    print "quantity: " + str(len(names))
    brands=[]#contains a set of brand
    details=[]#contains everything
    for i in range(len(names)):
        brands.append(names[i].text.split()[0])
        confi=get_confi('http://fptshop.com.vn' + href[i])
        #print 'http://fptshop.com.vn' + href[i]
        temp=[names[i].text]
        try:
            temp.extend(confi)
        except:
            temp.extend([None for j in range(9)])
        try:
            temp.append(price_filter(prices[i].text))
        except:
            temp.append(None)
        details.append(temp)
        print temp

    return details
#driver=webdriver.Chrome("F:\\Deadlines\\chromedriver.exe")
#get_phone_info('https://fptshop.com.vn/dien-thoai/nokia')

In [9]:
url = 'http://fptshop.com.vn/dien-thoai'
html_text = requests.get(url).text
tree = BeautifulSoup(html_text)
driver = webdriver.Chrome("F:\\Deadlines\\chromedriver.exe")
menu = tree.find(class_='fshop-lfter-lbot dropdown-menu listfilternomal')
side=menu.find_all('a')
categories=[]
for ele in side:
    categories.append(ele['data-value'])
del categories[0]
#print categories
details=[]
for category in categories:
    try:
        details.extend(get_phone_info('http://fptshop.com.vn/dien-thoai/' + category))
    except:
        continue
print 'DONE'

C:\Anaconda2\lib\site-packages\bs4\__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


[<h3>iPhone 7 Plus 256GB</h3>, <h3>iPhone 7 Plus 128GB</h3>, <h3>iPhone 7 Plus 32GB</h3>, <h3> iPhone 7 256GB</h3>, <h3>iPhone 7 128GB PRODUCT RED</h3>, <h3>iPhone 7 128GB</h3>, <h3>iPhone 7 32GB</h3>, <h3>iPhone 6s 64GB</h3>, <h3>iPhone 6s 32GB</h3>, <h3>iPhone 6s Plus 16GB</h3>, <h3>iPhone 6 Plus 16GB</h3>, <h3>iPhone 6 32GB (2017)</h3>, <h3>iPhone 5s 16GB</h3>, <h3>iPhone 7 Plus 256GB PRODUCT RED</h3>, <h3>iPhone 7 Plus 128GB PRODUCT RED</h3>, <h3>iPhone 7 256GB PRODUCT RED</h3>, <h3>iPhone 6s Plus 32GB</h3>, <h3>iPhone 6s 16GB</h3>]
quantity: 18
[u'iPhone 7 Plus 256GB', u'1920 x 1080', u'5.5 inch', u'12.0 MP', u'7.0 MP', u'2.3 GHz', u'3 GB', u'256GB', u'2900 mAh', None, u'25.999.000']
[u'iPhone 7 Plus 128GB', u'1334 x 750', u'5.5 inch', u'12.0 MP', u'7.0 MP', u'2.3 GHz', u'3 GB', u'128 GB', u'2900 mAh', 134, u'23.999.000']
[u'iPhone 7 Plus 32GB', u'1920 x 1080', u'5.5 inch', u'12.0 MP', u'7.0 MP', u'2.3 GHz', u'3 GB', u'32 GB', u'2900 mAh', 134, u'21.999.000']
[u' iPhone 7 256GB', 

In [10]:
import numpy as np
import pandas as pd
df = pd.DataFrame(details, columns=['Phone Name','resolution','screen size','rear camera','front camera','CPU', 'RAM', 'Internal storage','Battery capacity','Days','Price'])
df

,Phone Name,resolution,screen size,rear camera,front camera,CPU,RAM,Internal storage,Battery capacity,Days,Price
0,iPhone 7 Plus 256GB,1920 x 1080,5.5 inch,12.0 MP,7.0 MP,2.3 GHz,3 GB,256GB,2900 mAh,NaN,25.999.000
1,iPhone 7 Plus 128GB,1334 x 750,5.5 inch,12.0 MP,7.0 MP,2.3 GHz,3 GB,128 GB,2900 mAh,134.0,23.999.000
2,iPhone 7 Plus 32GB,1920 x 1080,5.5 inch,12.0 MP,7.0 MP,2.3 GHz,3 GB,32 GB,2900 mAh,134.0,21.999.000
3,iPhone 7 256GB,1334 x 750,4.7 inch,12.0 MP,7.0 MP,2.3 GHz,2 GB,256GB,1960 mAh,120.0,21.999.000
4,iPhone 7 128GB PRODUCT RED,1334 x 750,4.7 inch,12.0 MP,7.0 MP,2.3 GHz,2 GB,128 GB,1960 mAh,23.0,19.999.000
5,iPhone 7 128GB,1334 x 750,4.7 inch,12.0 MP,7.0 MP,2.3 GHz,2 GB,128 GB,1960 mAh,7.0,19.999.000
6,iPhone 7 32GB,1334 x 750,4.7 inch,12.0 MP,7.0 MP,2.3 Ghz,2 GB,32 GB,1960 mAh,1.0,17.999.000
7,iPhone 6s 64GB,1334 x 750,4.7 inch,12.0 MP,5.0 MP,None,2 GB,64 GB,None,134.0,15.999.000
8,iPhone 6s 32GB,1334 x 750,4.7 inch,12.0 MP,5.0 MP,1.8 GHz,2 GB,32 GB,None,8.0,14.999.000
9,iPhone 6s Plus 16GB,1080 x 1920,5.5 inch,12.0 MP,5.0 MP,1.8 GHz,2 GB,16 GB,None,1.0,13.999.000


In [11]:
print categories

['apple-iphone', 'sony', 'samsung', 'htc', 'nokia', 'lg', 'asus', 'lenovo', 'fpt', 'gionee', 'huawei', 'mobell', 'mobiistar', 'oppo', 'philips', 'wing', 'masstel', 'pantech', 'obi-worldphone', 'motorola', 'coolpad', 'oneplus', 'meizu', 'vivo', 'wiko', 'tecno', 'itel']


In [12]:
df.to_csv('FPT2.csv', encoding='utf-8')

def get_num_day(url):
    while(True):
        try:
            driver.find_element_by_link_text('»»').click()
        except:
            break
    tree = BeautifulSoup(driver.page_source)
    date=tree.find_all(class_='comment-date')
    print day_filter(date[-1].text)
    return day_filter(date[-1].text)
driver = webdriver.Chrome("F:\\Deadlines\\chromedriver.exe")
driver.get('http://fptshop.com.vn/dien-thoai/iphone-7-plus-256gb')
get_num_day('http://fptshop.com.vn/dien-thoai/iphone-7-plus-256gb')

url='http://fptshop.com.vn/dien-thoai/iphone-7-plus-256gb'
html_text = requests.get(url).text
tree = BeautifulSoup(html_text)
dienthoai=tree.find(class_='more-specification')
a=[s.extract() for s in dienthoai('label')]
b=[s.extract() for s in dienthoai('span')]

for i in range(len(a)):
    print a[i],b[i]
print find(u'Quay phim',a)